In [154]:
%matplotlib inline

# Build the Neural Network
Neural networks -- 'layers' of operations.  Basically each layer is a model, with outputs of one layer acting as inputs to the next.

`torch.nn` namespace provides all the building blocks you need to build your own neural network.  Every module in PyTorch subclasses the `nn.Module`.  A neural network is a module itself that consists of other modules (layers).  This nested structure allows for building and managing complex architectures easily.

Will use a NN to classify the FashionMNIST dataset now.

In [155]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## Get Device for Training
We want to be able to use the GPU if available, so check if it's available and set 'device' to gpu if so, else cpu.

In [156]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device.')

Using cpu device.


## Define the NN class
We define the neural network by subclassing `nn.Module`, and initialize the layers in `__init__`.  Every `nn.Module` subclass implements the operations on input data in hte `forward` method.

In [157]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

We create an instance of `NeuralNetwork`, move it to `device`, and print its structure.

In [158]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


To use this model, we pass the input data directly to the object.  This executes the model's `forward` along with some background operations.  __Do not call `model.forward()`__ directly!

Calling hte model on the input returns a 10-dimensional tensor wit raw predicted values for each class.  We get the prediction probabilities by passin it through an instance of the `nn.Softmax` module (why not just divide by the number of objects?)

In [159]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits) #huh?
y_pred = pred_probab.argmax(1)
print(f'Predicted class: {y_pred}')

Predicted class: tensor([1])


---------------------------------------

## Model Layers
Let's break down the layers in the FashionMNIST model.  To illustrate it, we take a sample minibatch of 3 images of size 28x28 and see what happens to it as we pass it through the network.

In [160]:
input_image = torch.rand(3,28,28)
print(input_image.shape)

torch.Size([3, 28, 28])


### nn.Flatten
We initialize the `nn.Flatten` layer to convert each 2D 28x28 image into a contiguous array of 784 pixel values (minibatch dimension @dim=0 is maintained).

In [161]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.shape)

torch.Size([3, 784])


### nn.Linear
The `linear layer` is a module that applies a linear transformation on the input using its stored weights and biases.

Note that here we go to 20 features in the hidden layer, rather than 512

In [162]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.shape)

torch.Size([3, 20])


### nn.RelU
Non-linear activations are what create the complex mappings between inputs and outputs (linear ones can, too!).  They are applied after the linear transformations to help neural networks learn non-linear phenomena.

In this model we use `nn.ReLU` -- basically if the input is less than zero the output is 0, otherwise the input is the output -- between our linear layers, but there's other activations to introduce non-linearity as well.

In [163]:
print(f'Before ReLU: {hidden1}\n\n')
hidden1 = nn.ReLU()(hidden1)
print(f'After ReLU: {hidden1}')

Before ReLU: tensor([[ 0.6389, -0.0292,  0.1809,  0.0834,  0.4015,  0.1838,  0.2065,  0.5643,
          0.4094, -0.0710, -0.1800,  0.0794,  0.0536, -0.0173, -0.6627, -0.4817,
         -0.2941, -0.1998, -0.6658,  0.0839],
        [ 0.4250,  0.0768,  0.0630, -0.1700,  0.6600, -0.0438,  0.5047,  0.1025,
          0.2629, -0.3986, -0.3077, -0.3239,  0.5094, -0.1489, -0.2294, -0.4071,
         -0.1764, -0.3255, -0.5530,  0.5317],
        [ 0.4202, -0.1688,  0.1328, -0.1579,  0.6434,  0.0483,  0.4223,  0.5331,
          0.0558, -0.2201, -0.4015, -0.2045,  0.3494, -0.0631, -0.4509, -0.4798,
          0.1360, -0.4389, -0.4496,  0.0095]], grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.6389, 0.0000, 0.1809, 0.0834, 0.4015, 0.1838, 0.2065, 0.5643, 0.4094,
         0.0000, 0.0000, 0.0794, 0.0536, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0839],
        [0.4250, 0.0768, 0.0630, 0.0000, 0.6600, 0.0000, 0.5047, 0.1025, 0.2629,
         0.0000, 0.0000, 0.0000, 0.5094, 0.0000, 0.000

### nn.Sequential
`nn.Sequential` is an ordered container of modules.  The data is passed through all the modules in the same order as defined.  You can use sequential containers to put together a quick network like `seq_modules`.

In [164]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

### nn.Softmax
The last linear layer of the neural network returns `logits` -- raw values in $[-\infty, \infty]$ -- which are passed to the `nn.Softmax` module.  The logits are scaled to values [0, 1] representing the model's predicted probabilities for each class. `dim` parameter indicates the dimension along which the values must sum to 1.

In [165]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

## Model Parameters
Many layers inside a neural network are *parameterized*, i.e. have associated weights and biases that are optimized during training.  Subclassing `nn.Module` automatically tracks all fields defined inside your model object, and makes all parameters accessible using your model's `parameters()` or `named_parameters()` methods.

In this example, we iterate over each parameter, and print its size and a preview of its values.

In [166]:
print('Model structure: ', model, '\n\n')

for name, param in model.named_parameters():
    print(f'Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n')

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0270, -0.0225,  0.0102,  ..., -0.0032, -0.0114, -0.0020],
        [-0.0051, -0.0341, -0.0073,  ...,  0.0330, -0.0321,  0.0310]],
       grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0164,  0.0192], grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0162,  0.0099,  0.0400,  ...,  0.0088,  0.0083,  0.0340],
        [ 0.0342,  0.0042, -0.0130,  ...,  0.0397,  0.0199,  0.0239]],
       grad_fn=<SliceBackward>) 

Layer: linear_relu_